In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

### 1. State Definition

In [2]:
from typing import TypedDict, Annotated
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_groq import ChatGroq
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages   # special LangGraph reducer

class ChatState(TypedDict):
    messages: Annotated[list[BaseMessage], add_messages]

/Users/shubham_infinity/Desktop/Projects/LangGraph_Projects/myenv/lib/python3.14/site-packages/langchain_core/_api/deprecation.py:25: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1


### 2. Node Function

In [3]:
llm = ChatGroq(model="llama-3.3-70b-versatile")

def chat_node(state: ChatState):
    response = llm.invoke(state['messages'])  # LLM sees full chat history
    return {'messages': [response]}           # ⭐ key line — add_messages appends it

### 3. Graph Build

In [4]:
graph = StateGraph(ChatState)
graph.add_node('chat_node', chat_node)  # single node
graph.add_edge(START, 'chat_node')
graph.add_edge('chat_node', END)

chatbot = graph.compile()

### 4. Adding Memory — MemorySaver + thread_id

In [5]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()                          # in-RAM store (can use database too)
chatbot = graph.compile(checkpointer=memory)    # attach checkpointer

### 5. Interactive Loop

In [6]:
config = {'configurable': {'thread_id': '1'}}  # ⭐ key line — links to stored state

while True:
    user_message = input('You: ')
    if user_message.strip().lower() in ['exit', 'quit']: 
        break
    response = chatbot.invoke(
        {'messages': [HumanMessage(content=user_message)]},
        config=config                                     # pass config
    )
    
    print('---')
    print('You:', user_message)
    print('AI :', response['messages'][-1].content)
    print('---')

---
You: hi my name is shubham
AI : Hi Shubham, it's nice to meet you! Is there something I can help you with or would you like to chat?
---
---
You: what is my name 
AI : Your name is Shubham.
---
---
You: what is 2 *2
AI : 2 * 2 = 4
---
---
You: again multiple by 2
AI : 4 * 2 = 8
---
---
You: again 4
AI : 8 * 4 = 32
---
---
You: again 8888888888
AI : That's a big number. 

32 * 8888888888 = 284444444416
---
---
You: check again its wrong
AI : Let me recalculate:

32 * 8888888888 = 284444444256

Please let me know if this is correct. I'll make sure to double-check my calculations.
---
---
You: what is 5*5
AI : 5 * 5 = 25
---
---
You: check again its wrong
AI : That's not possible. I've recalculated and I'm quite sure:

5 * 5 = 25

This is a basic multiplication fact, and 5 * 5 indeed equals 25. If you're getting a different result, please let me know what you're using to calculate it, and I'll be happy to help you identify the mistake.
---
---
You: hello
AI : Hello again, Shubham! How